To perform item and time dimensional inventory analytics based on the given schema, we can utilize SQL queries to gather insights. Here are a few examples of analytics that can be performed:

Database connector class. It will be used to coneect to database and do different types of olap queries on data.

In [1]:
import psycopg2
import pandas as pd
import psycopg2.extras
import matplotlib.pyplot as plt
import numpy as np


class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user="dhiman",
                                           password="dhiman1234",
                                           host="127.0.0.1",
                                           port="5432")

    def getConnection(self):
        return self.connection

### 1. Total inventory value by item:

In [2]:
connection = PostgresConnection().getConnection()
cursor = connection.cursor()
select_stmt = '''   SELECT fact_table.item_key, item_name, SUM(quantity * fact_table.unit_price) AS total_inventory_value
                    FROM fact_table
                    JOIN item_dim ON fact_table.item_key = item_dim.item_key
                    GROUP BY fact_table.item_key, item_name;   '''

# print(select_stmt)

cursor.execute(select_stmt)
records = cursor.fetchall()

df = pd.DataFrame(list(records))
print(df.head(len(df.index)))

cursor.close()
connection.close()

          0                                      1          2
0    I00001             A&W Root Beer - 12 oz cans   271664.5
1    I00002        A&W Root Beer Diet - 12 oz cans  156478.50
2    I00003          Barq's Root Beer - 12 oz cans  154764.00
3    I00004                       Cherry Coke 12oz  153765.00
4    I00005               Cherry Coke Zero 12 pack  152151.75
..      ...                                    ...        ...
259  I00260                   Napkins Square Lunch   348795.0
260  I00261                 POM 2 ply paper towels   746064.0
261  I00262                Strong Everyday Napkins   315798.0
262  I00263                   Advil 2 pill packets   323764.0
263  I00264  Tylenol Extra Strength 2 pill packets   340350.0

[264 rows x 3 columns]


### 2. Top-selling items by quantity:

In [3]:
connection = PostgresConnection().getConnection()
cursor = connection.cursor()
select_stmt = '''   SELECT fact_table.item_key, item_name, SUM(quantity) AS total_quantity_sold
                    FROM fact_table
                    JOIN item_dim ON fact_table.item_key = item_dim.item_key
                    GROUP BY fact_table.item_key, item_name
                    ORDER BY total_quantity_sold DESC
                    LIMIT 50;   '''

# print(select_stmt)

cursor.execute(select_stmt)
records = cursor.fetchall()

df = pd.DataFrame(list(records))
print(df.head(len(df.index)))

cursor.close()
connection.close()

         0                                           1      2
0   I00015                     Diet Pepsi - 12 oz cans  23969
1   I00203        Nat.Valley PeanutButter Protein Bars  23958
2   I00230         Nabisco Classic Mix ccooki, cracker  23852
3   I00023                          Pepsi - 12 oz cans  23802
4   I00061                               Red Bull 12oz  23740
5   I00246                Cascade Gel Packs Dishwasher  23648
6   I00001                  A&W Root Beer - 12 oz cans  23623
7   I00110                       Starbuck's Pike Place  23621
8   I00229          Kellogg's Cereals Assortment Pack   23583
9   I00209      Quaker Instant Oatmeal Maple Brn Sugar  23560
10  I00135      Chinet Comfort Hot Cups with Lids 16oz  23516
11  I00115         K Cups Daily Chef Columbian Supremo  23498
12  I00072                     Gatorade Variety 20 oz   23474
13  I00219             Wonderful Pistachios (in shell)  23454
14  I00257               Large Trash Bags Black 33 gal  23435
15  I001

### 3. Monthly sales trend:

In [4]:
connection = PostgresConnection().getConnection()
cursor = connection.cursor()
select_stmt = '''   SELECT time_year, time_month, SUM(total_price) AS total_sales
                    FROM fact_table
                    JOIN time_dim ON fact_table.time_key = time_dim.time_key
                    GROUP BY time_year, time_month
                    ORDER BY time_year, time_month;   '''

# print(select_stmt)

cursor.execute(select_stmt)
records = cursor.fetchall()

df = pd.DataFrame(list(records))
print(df.head(len(df.index)))

cursor.close()
connection.close()

       0   1           2
0   2014   1   496549.25
1   2014   2  1122547.00
2   2014   3  1262435.75
3   2014   4  1245142.50
4   2014   5  1268770.50
..   ...  ..         ...
80  2020   9  1232639.75
81  2020  10  1319655.75
82  2020  11  1200248.75
83  2020  12  1346884.75
84  2021   1   883772.25

[85 rows x 3 columns]


### 4. Average unit price by item:

In [5]:
connection = PostgresConnection().getConnection()
cursor = connection.cursor()
select_stmt = '''   SELECT item_key, item_name, AVG(unit_price) AS average_unit_price
                    FROM item_dim
                    GROUP BY item_key, item_name;  '''

# print(select_stmt)

cursor.execute(select_stmt)
records = cursor.fetchall()

df = pd.DataFrame(list(records))
print(df.head(len(df.index)))

cursor.close()
connection.close()

          0                                       1                    2
0    I00182       Reese's Pieces Peanutbutter Candy  15.0000000000000000
1    I00101    French Vanilla International Delight  15.0000000000000000
2    I00237      Breath Savers Peppermints 12/roll   16.0000000000000000
3    I00085      Tropicana Orange Juice 10 oz 100%   18.0000000000000000
4    I00154                       Chex Mix Original  15.0000000000000000
..      ...                                     ...                  ...
259  I00261                  POM 2 ply paper towels  33.0000000000000000
260  I00173           Hershey Factory Favorites Mix  19.0000000000000000
261  I00209  Quaker Instant Oatmeal Maple Brn Sugar  13.0000000000000000
262  I00242           Trident Original Flavor 18/pk  12.0000000000000000
263  I00051                      Monster Java 15 oz  23.0000000000000000

[264 rows x 3 columns]


### 5. Inventory turnover rate by item:

In [6]:
connection = PostgresConnection().getConnection()
cursor = connection.cursor()
select_stmt = '''   SELECT fact_table.item_key, item_name, SUM(quantity) AS total_quantity_sold, AVG(quantity) AS average_quantity_sold
                    FROM fact_table
                    JOIN item_dim ON fact_table.item_key = item_dim.item_key
                    GROUP BY fact_table.item_key, item_name;  '''

# print(select_stmt)

cursor.execute(select_stmt)
records = cursor.fetchall()

df = pd.DataFrame(list(records))
print(df.head(len(df.index)))

cursor.close()
connection.close()

          0                                      1      2                   3
0    I00001             A&W Root Beer - 12 oz cans  23623  6.0774376125546694
1    I00002        A&W Root Beer Diet - 12 oz cans  23182  6.0464267083985394
2    I00003          Barq's Root Beer - 12 oz cans  22928  6.0672135485578195
3    I00004                       Cherry Coke 12oz  22780  5.9790026246719160
4    I00005               Cherry Coke Zero 12 pack  22541  5.9679639925867090
..      ...                                    ...    ...                 ...
259  I00260                   Napkins Square Lunch  23253  5.9485801995395242
260  I00261                 POM 2 ply paper towels  22608  6.0823244552058111
261  I00262                Strong Everyday Napkins  22557  5.9658820417878868
262  I00263                   Advil 2 pill packets  23126  6.0650406504065041
263  I00264  Tylenol Extra Strength 2 pill packets  22690  6.0313662945241893

[264 rows x 4 columns]
